In [3]:
!pip install selenium beautifulsoup4 pandas chromedriver-autoinstaller

^C


You should consider upgrading via the 'c:\users\daniy\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


  Using cached selenium-4.26.1-py3-none-any.whl (9.7 MB)
  Using cached beautifulsoup4-4.12.3-py3-none-any.whl (147 kB)
  Using cached pandas-2.2.3-cp39-cp39-win_amd64.whl (11.6 MB)
  Using cached chromedriver_autoinstaller-0.6.4-py3-none-any.whl (7.6 kB)
  Using cached trio-0.27.0-py3-none-any.whl (481 kB)
  Using cached certifi-2024.8.30-py3-none-any.whl (167 kB)
  Using cached trio_websocket-0.11.1-py3-none-any.whl (17 kB)
  Using cached urllib3-2.2.3-py3-none-any.whl (126 kB)
  Using cached websocket_client-1.8.0-py3-none-any.whl (58 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl (10 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl (29 kB)
  Using cached idna-3.10-py3-none-any.whl (70 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)
  Using cached attrs-24.2.0-py3-none-any.whl (63 kB)
  Using cached cffi-1.17.1-cp39-cp39-win_amd64.whl (181 kB)
  Using cached pycparser-2.22-py3-none-any.whl (117 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl (2

In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import pandas as pd
import sys
import logging
import random
import chromedriver_autoinstaller

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Set up Selenium WebDriver
def setup_driver():
    chromedriver_autoinstaller.install()
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')
    driver = webdriver.Chrome(options=options)
    driver.set_page_load_timeout(60)  # Add timeout here
    return driver

# Extract all links from a page
def get_all_links(driver, base_url):
    links = set()
    elements = driver.find_elements(By.TAG_NAME, 'a')
    for element in elements:
        href = element.get_attribute('href')
        if href and href.startswith(base_url):  # Only internal links
            links.add(href)
    return links

# Scrape data from a single page
def scrape_page(driver, url, retries=3):
    for attempt in range(retries):
        try:
            driver.get(url)
            WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            return soup.get_text(strip=True)
        except Exception as e:
            logging.error(f"Attempt {attempt + 1} failed for {url}: {e}")
            time.sleep(2)  # Wait before retrying
    return ""

# Recursive function to crawl and scrape
def crawl_and_scrape(driver, base_url, visited, data, max_depth, current_depth=0):
    if base_url in visited or current_depth > max_depth:
        return
    visited.add(base_url)

    logging.info(f"Scraping: {base_url}")
    page_data = scrape_page(driver, base_url)
    data.append({'url': base_url, 'content': page_data})

    links = get_all_links(driver, base_url)
    for link in links:
        time.sleep(random.uniform(1, 3))  # Random delay to avoid overwhelming the server
        crawl_and_scrape(driver, link, visited, data, max_depth, current_depth + 1)

# Main function to start scraping
def main():
    # Set default values
    base_url = "https://lhr.nu.edu.pk/"  # Default URL
    max_depth = 3  # Default maximum depth for crawling

    # Allow user input
    base_url = input("Enter the base URL (default is 'https://lhr.nu.edu.pk/'): ") or base_url
    max_depth_input = input("Enter the maximum depth (default is 3): ")
    if max_depth_input.isdigit():
        max_depth = int(max_depth_input)

    driver = setup_driver()

    visited = set()
    data = []

    try:
        crawl_and_scrape(driver, base_url, visited, data, max_depth)
    finally:
        driver.quit()

    # Save the data
    df = pd.DataFrame(data)
    df.to_csv('website_data.csv', index=False)
    logging.info("Data saved to 'website_data.csv'")

if __name__ == "__main__":
    main()


2024-11-16 23:12:46,876 - INFO - Chromedriver is already installed.
2024-11-16 23:12:49,604 - INFO - Scraping: https://lhr.nu.edu.pk/
2024-11-16 23:13:09,745 - INFO - Scraping: https://lhr.nu.edu.pk/newsDetails/512
2024-11-16 23:13:13,944 - INFO - Scraping: https://lhr.nu.edu.pk/newsDetails/512#
2024-11-16 23:13:19,153 - INFO - Scraping: https://lhr.nu.edu.pk/research/
2024-11-16 23:13:24,441 - INFO - Scraping: https://lhr.nu.edu.pk/research/#
2024-11-16 23:13:29,354 - INFO - Scraping: https://lhr.nu.edu.pk/alumniaward/
2024-11-16 23:13:51,685 - INFO - Scraping: https://lhr.nu.edu.pk/alumniaward/#
2024-11-16 23:13:57,261 - INFO - Scraping: https://lhr.nu.edu.pk/ss/programDetails/MS%20(English%20Language%20Teaching)
2024-11-16 23:14:01,718 - INFO - Scraping: https://lhr.nu.edu.pk/ss/programDetails/MS%20(English%20Language%20Teaching)#
2024-11-16 23:14:05,957 - INFO - Scraping: https://lhr.nu.edu.pk/newsDetails/500
2024-11-16 23:14:09,735 - INFO - Scraping: https://lhr.nu.edu.pk/newsDeta

In [8]:
import pandas as pd
import re
from sentence_transformers import SentenceTransformer, util

# Step 1: Enhanced Data Cleaning
def clean_text(text):
    """
    Clean text by removing unnecessary spaces, special characters,
    converting to lowercase, and splitting concatenated words.
    """
    text = str(text).lower()  # Convert to lowercase
    text = re.sub(r'(\w)([A-Z])', r'\1 \2', text)  # Separate camelCase words
    text = re.sub(r'([a-z])([A-Z])', r'\1 \2', text)  # Separate PascalCase words
    text = re.sub(r'(\w)([^\w\s])', r'\1 ', text)  # Split words stuck to punctuation
    text = re.sub(r'[^\w\s]', '', text)  # Remove remaining special characters
    text = re.sub(r'\s+', ' ', text)  # Remove extra whitespaces
    return text.strip()

def clean_url(url):
    """
    Simplify URL by removing fragments and query parameters.
    """
    url = re.sub(r'#.*$', '', url)  # Remove fragments (#)
    url = re.sub(r'\?.*$', '', url)  # Remove query parameters (?)
    return url.strip()

# Load dataset
df = pd.read_csv("website_data.csv")

# Apply content cleaning
df['content'] = df['content'].apply(clean_text)

# Apply URL cleaning
df['url'] = df['url'].apply(clean_url)

# Remove rows with empty content
df = df[df['content'].str.strip() != ""]

# Drop duplicate URLs
df = df.drop_duplicates(subset='url', keep='first').reset_index(drop=True)

# Save the cleaned dataset for future use
df.to_csv("cleaned_website_data.csv", index=False)
print("Cleaned dataset saved.")

# Step 2: Load SentenceTransformer model and Encode Content
model = SentenceTransformer('all-MiniLM-L6-v2')
print("Model loaded.")

# Create content embeddings
content_embeddings = model.encode(df['content'], convert_to_tensor=True)
print("Embeddings created.")

Cleaned dataset saved.
Model loaded.
Embeddings created.


In [11]:
!pip install sentence-transformers


  Using cached sentence_transformers-3.3.0-py3-none-any.whl (268 kB)
  Using cached tqdm-4.67.0-py3-none-any.whl (78 kB)
  Using cached transformers-4.46.2-py3-none-any.whl (10.0 MB)
  Using cached torch-2.5.1-cp39-cp39-win_amd64.whl (203.0 MB)
  Using cached huggingface_hub-0.26.2-py3-none-any.whl (447 kB)
  Using cached pillow-11.0.0-cp39-cp39-win_amd64.whl (2.6 MB)
  Using cached scikit_learn-1.5.2-cp39-cp39-win_amd64.whl (11.0 MB)
  Using cached filelock-3.16.1-py3-none-any.whl (16 kB)
  Using cached requests-2.32.3-py3-none-any.whl (64 kB)
  Using cached fsspec-2024.10.0-py3-none-any.whl (179 kB)
  Using cached PyYAML-6.0.2-cp39-cp39-win_amd64.whl (162 kB)
  Using cached networkx-3.2.1-py3-none-any.whl (1.6 MB)
  Using cached jinja2-3.1.4-py3-none-any.whl (133 kB)
  Using cached sympy-1.13.1-py3-none-any.whl (6.2 MB)
  Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
  Using cached tokenizers-0.20.3-cp39-none-win_amd64.whl (2.4 MB)
  Using cached regex-2024.11.6-cp39-cp39-win_a

ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'c:\\users\\daniy\\appdata\\local\\programs\\python\\python39\\Lib\\site-packages\\scipy\\interpolate\\_ndgriddata.py'
Consider using the `--user` option or check the permissions.

You should consider upgrading via the 'c:\users\daniy\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [23]:
!python -m pip install --upgrade pip

  Attempting uninstall: pip
    Found existing installation: pip 21.1.1
    Uninstalling pip-21.1.1:
      Successfully uninstalled pip-21.1.1


In [10]:
# Step 3: Define Chatbot Function
def chatbot_response(query):
    """
    Generate a response for the user query based on cosine similarity.
    """
    # Clean the user query
    query = clean_text(query)
    
    # Generate embedding for the query
    query_embedding = model.encode(query, convert_to_tensor=True)
    
    # Calculate cosine similarity
    cosine_scores = util.cos_sim(query_embedding, content_embeddings)
    
    # Find the best match
    best_match_index = cosine_scores.argmax().item()
    best_match_score = cosine_scores[0, best_match_index].item()
    
    # If the match score exceeds the threshold, return the result
    if best_match_score > 0.4:  # Adjust threshold as needed
        best_match_content = df.iloc[best_match_index]['content']
        best_match_url = df.iloc[best_match_index]['url']
        return f"{best_match_content} \nFor more details, visit: {best_match_url}"
    else:
        return "I'm sorry, I couldn't find an answer to your question."

# Step 4: Run the Chatbot
if __name__ == "__main__":
    print("Chatbot is ready! Type 'exit' or 'quit' to end the conversation.")
    while True:
        user_input = input("You: ")
        print(f"User said: {user_input}") 
        if user_input.lower() in ['exit', 'quit']:
            print("Chatbot: Goodbye!")
            break
        response = chatbot_response(user_input)
        print(f"Chatbot: {response}")


Chatbot is ready! Type 'exit' or 'quit' to end the conversation.
User said: what is fast?
Chatbot: I'm sorry, I couldn't find an answer to your question.
User said: bs programs
Chatbot: cs hometel 042 111 128 128follow us homepageblock b faisal town lahore fast school of computingfast nu lahorefscfacultyprogramsresearchstudentsfypthesis resourcestimetablenews eventsfast school of computing programsfsc programsbachelors programsbs computer science bs data science bs software engineering masters programsms computer science ms data science ms software project management phd programsphd computer science important linkscampus homecampus aboutadmissionslibrarysupport and servicesslateflexdepartmentsfast school of managementfast school of computingdepartment of electrical engineeringdepartment of civil engineeringdepartment of sciences and humanities 2020 copyright lhr nu edu pk 
For more details, visit: https://lhr.nu.edu.pk/fsc/programsHome/
User said: admissions open?
Chatbot: fast nuces l